In [45]:
%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np
import tensorflow as tf
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

sys.path.insert(1, os.path.join(sys.path[0], ".."))
from reproduction.pipeline.load import load_data
from reproduction import analysis
import sklearn
from scipy.stats import wasserstein_distance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
# open/closed cell data dir
CLOSEDATADIR="/home/tkurihana/scratch-midway2/clouds/"
CLOSEDS = "/closed.tfrecord"
OPENDATADIR="/home/tkurihana/scratch-midway2/clouds/"
OPENS = "/open.tfrecord"

# outputdata dir
OUTPUTDIR="/project/foster/clouds/output"
CENTERS = OUTPUTDIR+"/mod09cnn15b/encodings/m15b-enc.cluster_centres"
ENCODER_DEF = OUTPUTDIR+"/mod09cnn15b/encoder.json"
ENCODER_WEIGHTS = OUTPUTDIR+"/mod09cnn15b/encoder-585000.h5"
ENCODINGS = OUTPUTDIR+"/mod09cnn15b/encodings/m15b-enc"

In [3]:
centers = analysis.read_kmeans_centers(CENTERS)

In [4]:
k = KMeans(n_clusters=centers.shape[0], init=centers)

In [5]:
with open(ENCODER_DEF,"r") as f:
    encoder = tf.keras.models.model_from_json(f.read())
encoder.load_weights(ENCODER_WEIGHTS)

In [77]:
def _classifier(encoder, keyward, opendir, openfile, closeddir, closedfile, num_of_file):
    OPENS   = opendir+keyward+"_open"+openfile
    CLOSEDS = closeddir+keyward+"_close"+closedfile
    
    # TODO: debug
    def decode_tfrecord(OUTPUT_TFRECORD_NAME):
        count = 0
        imgs = []
        print(OUTPUT_TFRECORD_NAME)
        for record  in tf.python_io.tf_record_iterator(OUTPUT_TFRECORD_NAME):
            count += 1
            #xample = tf.train.Example()
            #xample.ParseFromString(record) # Read binary data
        #rint(example)
    
        print(count)
        #print(height, width, depth)
        return count
    # get file number
    no = decode_tfrecord(OPENS)
    nc = decode_tfrecord(CLOSEDS)
    print(no, nc)
    N  = num_of_file # min(no, nc) -1 # get less number 
    print(N)
    
    ## Repeat off and set reasonable n (patch) number
    opens = analysis.AEData(load_data(OPENS, encoder.input_shape[1:], repeat=False), n=N)
    closeds = analysis.AEData(load_data(CLOSEDS, encoder.input_shape[1:], repeat=False), n=N)  
    #  encpder 
    opens.add_encoder(encoder)
    closeds.add_encoder(encoder)
    #  cluster
    k.cluster_centers_ = centers
    #
    both = np.append(opens.encs,closeds.encs, axis=0)
    num_clusters = 10
    method = SpectralClustering(n_clusters=num_clusters, affinity="nearest_neighbors")
    clusters = method.fit_predict(both)
    o, c = clusters[:N], clusters[N:]
    #o = 1 ; c = 1
    return o, c

In [93]:
o, c = _classifier(encoder,'2017JAN03MOD09GA', OPENDATADIR, OPENS, CLOSEDATADIR, CLOSEDS, 15)

/home/tkurihana/scratch-midway2/clouds/2017JAN03MOD09GA_open/open.tfrecord
17
/home/tkurihana/scratch-midway2/clouds/2017JAN03MOD09GA_close/closed.tfrecord
77
17 77
15


OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext_31 = IteratorGetNext[output_shapes=[[32], [32,2], [32,128,128,7]], output_types=[DT_STRING, DT_INT64, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_31)]]

Caused by op 'IteratorGetNext_31', defined at:
  File "/home/tkurihana/.conda/envs/py3501/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tkurihana/.conda/envs/py3501/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tkurihana/.conda/envs/py3501/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tkurihana/.conda/envs/py3501/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/tkurihana/.conda/envs/py3501/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/home/tkuriha

In [86]:
keylist = ['2017JAN01MOD09GA', '2017JAN02MOD09GA', '2017JAN03MOD09GA', '2017JAN04MOD09GA']

In [87]:
# TODO: debug
def _decode_tfrecord(OUTPUT_TFRECORD_NAME):
        count = 0
        imgs = []
        print(OUTPUT_TFRECORD_NAME)
        for record  in tf.python_io.tf_record_iterator(OUTPUT_TFRECORD_NAME):
            count += 1
            #xample = tf.train.Example()
            #xample.ParseFromString(record) # Read binary data
        #rint(example)
    
        print(count)
        #print(height, width, depth)
        return count

In [92]:
# get file number
for i in range(4):
    opendir    = OPENDATADIR
    openfile   = OPENS
    closeddir  = CLOSEDATADIR
    closedfile = CLOSEDS
    keyward = keylist[i]

    _OPENS   = opendir+keyward+"_open"+openfile
    _CLOSEDS = closeddir+keyward+"_close"+closedfile
    no = _decode_tfrecord(_OPENS)
    nc = _decode_tfrecord(_CLOSEDS)
    print(no, nc)

/home/tkurihana/scratch-midway2/clouds/2017JAN01MOD09GA_open/open.tfrecord
38
/home/tkurihana/scratch-midway2/clouds/2017JAN01MOD09GA_close/closed.tfrecord
38
38 38
/home/tkurihana/scratch-midway2/clouds/2017JAN02MOD09GA_open/open.tfrecord
6
/home/tkurihana/scratch-midway2/clouds/2017JAN02MOD09GA_close/closed.tfrecord
29
6 29
/home/tkurihana/scratch-midway2/clouds/2017JAN03MOD09GA_open/open.tfrecord
17
/home/tkurihana/scratch-midway2/clouds/2017JAN03MOD09GA_close/closed.tfrecord
77
17 77
/home/tkurihana/scratch-midway2/clouds/2017JAN04MOD09GA_open/open.tfrecord
28
/home/tkurihana/scratch-midway2/clouds/2017JAN04MOD09GA_close/closed.tfrecord
5
28 5
